In [12]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [13]:
# 1. 數據加載與處理
def load_and_preprocess_data(file_path):
    # 讀取資料
    df = pd.read_csv(file_path)

    # 民國年轉西元年
    def convert_date(x):
        parts = x.split('/')
        year = int(parts[0]) + 1911  # 民國年轉換
        return f"{year}-{parts[1]}-{parts[2]}"

    df['日期'] = df['日期'].apply(convert_date)
    df['日期'] = pd.to_datetime(df['日期'], format='%Y-%m-%d')

    # 添加時間特徵
    df['天數'] = (df['日期'] - df['日期'].min()).dt.days
    df['月'] = df['日期'].dt.month
    df['日'] = df['日期'].dt.day
    df['星期'] = df['日期'].dt.weekday

    # 添加滯後特徵
    for i in range(1, 8):
        df[f'滯後{i}日平均價'] = df['平均價'].shift(i)

    # 創建目標變量: 7天後的平均價
    df['7日後平均價'] = df['平均價'].shift(-7)

    # 去除無效數據
    df = df.dropna()

    return df

# 創建序列數據
def create_sequences(data, target, time_steps=7):
    sequences = []
    labels = []
    for i in range(len(data) - time_steps):
        seq = data[i:i + time_steps]
        label = target[i + time_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [14]:
# 2. 訓練與驗證
def train_and_evaluate_model(df):
    # 分割數據集
    train_data = df[df['日期'] < datetime(2022, 1, 1)]
    val_data = df[(df['日期'] >= datetime(2022, 1, 1)) & (df['日期'] < datetime(2024, 1, 1))]
    test_data = df[df['日期'] >= datetime(2024, 1, 1)]

    # 特徵與目標
    features = ['天數', '月', '日', '星期'] + [f'滯後{i}日平均價' for i in range(1, 8)]
    X_train_raw, y_train_raw = train_data[features].values, train_data['7日後平均價'].values
    X_val_raw, y_val_raw = val_data[features].values, val_data['7日後平均價'].values
    X_test_raw = test_data[features].values

    # 創建序列數據
    time_steps = 7
    X_train, y_train = create_sequences(X_train_raw, y_train_raw, time_steps)
    X_val, y_val = create_sequences(X_val_raw, y_val_raw, time_steps)

    # 構建 LSTM 模型
    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(50, activation='relu'),
        Dense(1)  # 單輸出回歸
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])

    # 訓練模型
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

    return model, X_test_raw, test_data, time_steps

In [18]:
# 3. 預測與輸出
def predict_and_save_results(model, X_test_raw, test_data, output_path, time_steps):
    # 創建測試集序列
    X_test, _ = create_sequences(X_test_raw, np.zeros(len(X_test_raw)), time_steps)

    # 預測測試集
    test_data = test_data.iloc[time_steps:]  # 因序列有偏移
    test_data['預測價'] = model.predict(X_test).flatten()

    # 檢查是否有真實值可用於計算 RMSE
    if '7日後平均價' in test_data.columns:
        rmse = mean_squared_error(test_data['7日後平均價'], test_data['預測價'], squared=False)
        print(f"Test RMSE: {rmse}")
    else:
        print("Test dataset does not contain true values for RMSE calculation.")

    # 保存結果
    test_data[['日期', '預測價']].to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")

In [19]:
# 主程式
file_path = "data/蕹菜小葉_台北一日交易行情.csv"
output_path = "output.csv"

# 數據加載與處理
df = load_and_preprocess_data(file_path)

# 訓練與驗證
model, X_test_raw, test_data, time_steps = train_and_evaluate_model(df)

# 預測與保存結果
predict_and_save_results(model, X_test_raw, test_data, output_path, time_steps)

Epoch 1/50


c:\Users\KuanYi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 5346.5601 - mse: 5346.5601 - val_loss: 414.7999 - val_mse: 414.7999
Epoch 2/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 300.2823 - mse: 300.2823 - val_loss: 417.5424 - val_mse: 417.5424
Epoch 3/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 300.2234 - mse: 300.2234 - val_loss: 695.2653 - val_mse: 695.2653
Epoch 4/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 270.3436 - mse: 270.3436 - val_loss: 375.0981 - val_mse: 375.0981
Epoch 5/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 266.7713 - mse: 266.7713 - val_loss: 392.4877 - val_mse: 392.4877
Epoch 6/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 244.8295 - mse: 244.8295 - val_loss: 428.6190 - val_mse: 428.6190
Epoch 7/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 271.4823 - mse: 271.4823 - val_loss: 377.6987 - val_mse: 377.6987
Epoch 8/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 343.0724 - mse: 343.0724 - val_loss: 392.7419 - val_mse: 392.7419
E

C:\Users\KuanYi\AppData\Local\Temp\ipykernel_28564\104465750.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['預測價'] = model.predict(X_test).flatten()
c:\Users\KuanYi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
